<a href="https://colab.research.google.com/github/OnroerendErfgoed/scriptorium/blob/main/get_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Accessing a collection of resources

This notebook show how to access a collection of resources from the Agentschap Onroerend Erfgoed webservices. It assumes you have already mastered the techniques described in the [`get_resource` notebook](https://github.com/OnroerendErfgoed/scriptorium/blob/main/notebooks/get_resource.ipynb).

Again we'll be using the Inventaris Onroerend Erfgoed as our example, but the techniques in this notebook could also be applied to the Onroerend Erfgoed Beelbank or Archeologieportaal.

The basic idea for accessing a collection is very similar to accesing a resource, we GET a certain URL. But, this time the resources does not reprensent a single resource, but a collection of resources. This has a few consequences:

* When accessing a single resource, we receive an *object* that consists of attributes and their values. When accessing a collection, we receive a *list* of objects.
* When accessing a single resource, we get all the data on a single object. When accessing a collection, we get a summary of each object in the collection. To get all the data, we need to GET the full resource.
* When accessing a collection of resources, it's possible that the collection is to big to send to the client in one go. In this case we divide the collection in smaller pieces, often called *pages*.
* When accessing a collection, we often want to *filter* the objects that returned to us. Eg. we do not want the collection of all Erfgoedobject, but a smaller part such as the collection of *all Erfgoedobjecten in Antwerpen*.

# A simple example
Let's start with a simple example. We'll request the entire collection of Erfgoedobject and output the first page of results we receive.

We'll also output how many results there actually are. This information is sent back by the server in the `Content-Range` header. This header will contain a value like `items 0-9/124`. This means we are viewing resources 0 through 9 (the first ten) out of a total of 124 resources.


In [21]:
# Import requests library to contact the REST service
import requests
# Import IPython library to produce Markdown
import IPython

# Create a requests Session
session = requests.Session()

# Set the default Accept header to `application/json`
session.headers.update({'Accept': 'application/json'})

# Make a request and store the response
response = session.get(
    'https://inventaris.onroerenderfgoed.be/erfgoedobjecten'
)

# Remove the first 6 characters from the header (items )
content_range = response.headers['Content-Range'][6:]
# The part after the slash is the total number of results
number_of_results = content_range.split('/')[1]
# Split the part before the slash on `-`, before is the first result in this page
start_result = content_range.split('/')[0].split('-')[0].strip()
# Split the part before the slash on `-`, after is the last result in this page
end_result = content_range.split('/')[0].split('-')[1].strip()

# Turn the response's JSON data into a Python dictionary
data = response.json()

output = f"**Showing results {start_result} to {end_result} of {number_of_results}**\n\n"
for erfgoedobject in data:
  output += f"* {erfgoedobject['naam']} ({erfgoedobject['uri']})\n"
  output += f"\t* *Locatie:* {erfgoedobject['locatie_samenvatting']}\n"
  output += f"\t* *Korte beschrijving:* {erfgoedobject['korte_beschrijving']}\n"


IPython.display.Markdown(output)

**Showing results 0 to 9 of 92863**

* Nautil (https://id.erfgoed.net/erfgoedobjecten/99018)
	* *Locatie:* Onbepaald
	* *Korte beschrijving:* De Nautil is een bakdekkruiser met kofferdek uit 1927, gebouwd in Kaag (Nederland).
* P904 Sambre (https://id.erfgoed.net/erfgoedobjecten/99063)
	* *Locatie:* Onbepaald
	* *Korte beschrijving:* De P904 Sambre was een patrouilleschip van het Rijnsmaldeel.
* SeaSens (https://id.erfgoed.net/erfgoedobjecten/99087)
	* *Locatie:* Onbepaald
	* *Korte beschrijving:* De SeaSens is een Duitse kustvaarder die tussen 1939 en 1941 werd gebouwd.
* Avila (https://id.erfgoed.net/erfgoedobjecten/99091)
	* *Locatie:* Onbepaald
	* *Korte beschrijving:* De Avila is het motorjacht van wijlen koning Boudewijn. Het jacht wordt in het Koninklijk Museum van het Leger en de Krijgsgeschiedenis in Brussel bewaard.
* Maaike (https://id.erfgoed.net/erfgoedobjecten/99105)
	* *Locatie:* Onbepaald
	* *Korte beschrijving:* De Maaike (voorheen Odilia) werd in 1911 gebouwd als motorsleepboot op de werf van Steven Seijmonsbergen in Amsterdam.
* Willebroek (https://id.erfgoed.net/erfgoedobjecten/99107)
	* *Locatie:* Onbepaald
	* *Korte beschrijving:* De Willebroek werd in 1954 gebouwd op de Duitse scheepswerf Clausen in Oberwinter en diende op de Belgische kanalen.
* L'Enfant Gâté (https://id.erfgoed.net/erfgoedobjecten/99166)
	* *Locatie:* Onbepaald
	* *Korte beschrijving:* L’Enfant Gâté is een Chassiron C.F. van 30 voet gebouwd in 1967 op de werven van Edouard Richard in Marennes.
* Esmeralda (https://id.erfgoed.net/erfgoedobjecten/99187)
	* *Locatie:* Aalst (Oost-Vlaanderen)
	* *Korte beschrijving:* De Esmeralda is een motorspits die in 1941 op de scheepswerf Van Praet-Dansaert in Baasrode werd gebouwd.
* Heilige Geestkapel (https://id.erfgoed.net/erfgoedobjecten/84)
	* *Locatie:* 't Maegelijnplein 1, Kattestraat 32-34 (Aalst)
	* *Korte beschrijving:* De Heilige Geestkapel is een restant van het Godshuis Tafel van de Heilige Geest en klimt op tot een bouwfase van 1470. Gerestaureerd in 1893 en vanaf 1901 vergroot met twee traveeën naar het oosten en oprichting van een nieuwe sacristie onder leiding van architect Jules Goethals. Van de neogotische interieuraankleding vanaf 1909 getuigt nog een glasraam van Jozef Casier. Deel ingebouwde, rechthoekige kapel onder leien zadeldak met dakruitertje, bekroond met een gebogen naaldspits, bestaat uit een verankerde baksteenbouw met gebruik van zandsteen voor de afgeschuinde sokkel en afwerking der muuropeningen.
* Honegem - Solegem (https://id.erfgoed.net/erfgoedobjecten/135205)
	* *Locatie:* Aalst, Hofstade (Aalst), Erpe (Erpe-Mere), Lede (Lede)
	* *Korte beschrijving:* Het door de Molenbeek doorsneden gebied Honegem - Solegem omvat een grote verscheidenheid aan landschapselementen met een grote floristische verscheidenheid tot gevolg. Het landschap wordt gekenmerkt door een afwisseling van graslanden, akkers en bossen. Het veen in de Honegemdepressie werd als turf gewonnen. Aan de rand van het gebied komen enkele hoeven en een watermolen voor.


# Filtering the collection

In our first examples, we're requesting the entire collection, more than 90.000 resources. As can be imagined, this is generally not what we want and accessing the entire collection would be very time consuming. Quite often, we're only interested in a subset of the collection. We might only care about Erfgoedobjecten in a certain gemeente (municipality) or provincie (province) or Erfgoedobjecten of a certain type. 

To filter a collection, we pass it parameters. These are added to the URl after a question mark `?` and look like `<parameter_name>=<parameter_value>`. When there is more than one parameter, they get sepparated by an ampersand `&`. So, if we pass 2 parameters, we get `?parameter_one=value_one&paramerer_two=value_two`.

To know which parameters the server accepts, we consult the API documentation of the server. Generally these can be accessed at `/api_doc`, relative to the server root. Eg. https://inventaris.onroerenderfgoed.be/api_doc or https://beeldbank.onroerenderfgoed.be/api_doc

For our second example, we'll limit the results to Erfgoedobjecten that are built heritage (`discipline=2`), that have some form of bescherming (`rechtsgevolgen=beschermd`) and are situated in Ghent (`gemeente=44021`).

In [25]:
# Import requests library to contact the REST service
import requests
# Import IPython library to produce Markdown
import IPython

# Create a requests Session
session = requests.Session()

# Set the default Accept header to `application/json`
session.headers.update({'Accept': 'application/json'})

# Make a request and store the response
response = session.get(
    'https://inventaris.onroerenderfgoed.be/erfgoedobjecten',
    params = {
        'gemeente': 44021,
        'discipline': 2,
        'rechtsgevolgen': 'beschermd'
    }
)

# Remove the first 6 characters from the header (items )
content_range = response.headers['Content-Range'][6:]
# The part after the slash is the total number of results
number_of_results = content_range.split('/')[1]
# Split the part before the slash on `-`, before is the first result in this page
start_result = content_range.split('/')[0].split('-')[0].strip()
# Split the part before the slash on `-`, after is the last result in this page
end_result = content_range.split('/')[0].split('-')[1].strip()

# Turn the response's JSON data into a Python dictionary
data = response.json()

output = f"**Showing results {start_result} to {end_result} of {number_of_results}**\n\n"
for erfgoedobject in data:
  output += f"* {erfgoedobject['naam']} ({erfgoedobject['uri']})\n"
  output += f"\t* *Locatie:* {erfgoedobject['locatie_samenvatting']}\n"
  output += f"\t* *Korte beschrijving:* {erfgoedobject['korte_beschrijving']}\n"


IPython.display.Markdown(output)

**Showing results 0 to 9 of 1290**

* Hollandstellung (https://id.erfgoed.net/erfgoedobjecten/127073)
	* *Locatie:* Assenede, Oosteeklo (Assenede), Vrasene (Beveren), Lapscheure, Moerkerke (Damme), Eeklo (Eeklo), Ertvelde (Evergem), Gent (Gent), Lembeke (Kaprijke), Knokke, Westkapelle (Knokke-Heist), Zaffelare (Lochristi), Eksaarde (Lokeren), Maldegem, Middelburg (Maldegem), Sint-Gillis-Waas (Sint-Gillis-Waas), Sint-Laureins (Sint-Laureins), Sinaai (Sint-Niklaas), Kemzeke, Stekene (Stekene)
	* *Korte beschrijving:* Duitse verdedigingsstelling uit de Eerste Wereldoorlog, opgetrokken ten zuiden van de Belgisch-Nederlandse grens tussen Knokke en Vrasene.
* Kanaal De Lieve  (https://id.erfgoed.net/erfgoedobjecten/301119)
	* *Locatie:* Evergem (Evergem), Gent, Wondelgem (Gent), Ronsele, Vinderhoute, Waarschoot, Zomergem (Lievegem)
	* *Korte beschrijving:* De Lieve is een historisch binnenscheepvaartkanaal dat tussen 1251 en 1269 van Gent naar Damme, oorspronkelijk Aardenburg, gegraven werd. De bloeiperiode situeert zich in de 13de en 14de eeuw. In Gent, Evergem en Lievegem is het kanaal in stukken als open waterloop bewaard gebleven.  
* Drie burgerhuizen ontworpen door Geo Henderick (https://id.erfgoed.net/erfgoedobjecten/18151)
	* *Locatie:* Aaigemstraat 18-20, 24-26 (Gent)
	* *Korte beschrijving:* Ensemble van drie burgerhuizen in een sobere, functionele baksteenarchitectuur in navolging van de Amsterdamse School, naar ontwerp van architect Geo Henderick. De woningen kwamen tot stand volgens bouwaanvragen van 1927 (nummer 18) en 1928 (nummer 20, 22-26) in opdracht van Alexander Van Heuverswijn.
* Hoekcomplex ontworpen door Geo Henderick (https://id.erfgoed.net/erfgoedobjecten/18579)
	* *Locatie:* Aaigemstraat 28-34, Patijntjestraat 2-4 (Gent)
	* *Korte beschrijving:* Beeldbepalend hoekcomplex van twee meergezinswoningen in expressionistische baksteenarchitectuur in navolging van de Amsterdamse School, naar ontwerp van Geo Henderick volgens een bouwaanvraag van 1931 in opdracht van Alexander Van Heuverswijn. Appartementsgebouw van vier bouwlagen hoog onder een plat dak met een afgesnuite hoektravee gemarkeerd door twee hoger oplopende muurdammen, uitgewerkt als geometrisch vormgegeven sierstukken.
* Provinciaal Handels- en Taalinstituut (https://id.erfgoed.net/erfgoedobjecten/306487)
	* *Locatie:* Abdisstraat 52-56, Henleykaai 83-84 (Gent)
	* *Korte beschrijving:* Het Provinciaal Handels- en Taalinstituut (PHTI) is een scholencomplex dat gerealiseerd werd tussen 1959 en 1984 in verschillende fasen, steeds naar ontwerp van Jan Tanghe en Francis Serck.
* Berg van Barmhartigheid en woning van de beheerder (https://id.erfgoed.net/erfgoedobjecten/18808)
	* *Locatie:* Abrahamstraat 13-15 (Gent)
	* *Korte beschrijving:* De Bergen van Barmhartigheid werden gesticht door aartshertogen Albrecht en Isabella. Hofarchitect Wenceslas Cobergher kocht het zogenaamde Dondersteen, dat in 1620 werd gesloopt en in 1621 werd de nieuwe bouw voltooid. Monumentale lijstgevel van veertien traveeën en drie bouwlagen met twee parallelle zadeldaken. Eerste voorbeeld van Vlaamse barokstijl te Gent, opgetrokken uit baksteen verwerkt met zand- en hardsteen. Verankerde bakstenen achtergevel van zeventien traveeën en drie bouwlagen. Het 15de-eeuwse gebouw links ernaast werd aangepast en ingericht als woning voor de beheerder van de Berg van Barmhartigheid. Dit zandstenen gebouw omvat van acht traveeën en twee bouwlagen. Van het hoekpand met de Bonifantenstraat resteert een gekanteeld omheiningsmuurtje met een 17de-eeuws rondboogpoortje.

* Augustijnenklooster (https://id.erfgoed.net/erfgoedobjecten/18818)
	* *Locatie:* Academiestraat 1 (Gent)
	* *Korte beschrijving:* Vermoedelijk in 1295 vestiging van de eerste augustijnen in een huis genaamd "ter Capellen", gelegen in de wijk de Briel op de hoek van de Lange Steenstraat en de Geldmunt, volgens de traditie geschonken door Gerelmus Borluut en gelegen nabij een kapel, gewijd aan Sint-Stephanus, welke aanvankelijk dienst deed als kloosterkerk.
* Koninklijke Academie voor Schone Kunsten (https://id.erfgoed.net/erfgoedobjecten/18821)
	* *Locatie:* Academiestraat zonder nummer (Gent)
	* *Korte beschrijving:* In 1748-1751 werd de Academie gesticht, die zich aan het begin van de 19de eeuw vestigde in de gebouwen van het voormalige augustijnenklooster. Voorhofgevel van de westvleugel van 1737-1738: bepleisterde en beschilderde lijstgevel met zeven traveeën en drie bouwlagen onder schilddak, uitgevoerd in een sobere Lodewijk XIV-stijl. Het complex werd in 1827 uitgebreid met de noordvleugel naar ontwerp van Louis Roelandt en in 1870-71 met een oostvleugel, beide in neoclassicistische stijl.
* Miljoenenkwartier (https://id.erfgoed.net/erfgoedobjecten/301433)
	* *Locatie:* Achilles Musschestraat, Congreslaan, Diksmuidestraat, Fleurusstraat, Jemappesstraat, Kortrijksesteenweg, Krijgslaan, Onafhankelijkheidslaan, Paul de Smet de Naeyerplein, Sint-Pietersaalststraat, Vaderlandstraat, Vrijheidslaan, Woeringenstraat (Gent)
	* *Korte beschrijving:* Het Miljoenenkwartier is een residentiële wijk die tijdens het interbellum werd ontwikkeld op de gronden van de Wereldtentoonstelling van 1913. Vooral de gegoede burgerij bouwde hier en de wijk werd een staalkaart van de toen gangbare bouwstijlen.
* Appartementsgebouw en burgerhuizen ontworpen door Jules Lippens (https://id.erfgoed.net/erfgoedobjecten/18539)
	* *Locatie:* Achilles Musschestraat 1-2, 3, Onafhankelijkheidslaan 1-6 (Gent)
	* *Korte beschrijving:* Hoek- en rijhuis samen met de aangrenzende panden in de Achilles Musschestraat ontworpen door architect J. Lippens in 1930. Verschillende panden in zakelijke baksteenarchitectuur met drie bouwlagen en plat dak, voorzien van voortuintjes afgesloten door een ijzeren hek. Opmerkelijk appartementsgebouw op de hoek met een op de Nederlandse architectuur geïnspireerde compositie met verspringende gevelvlakken van erkers en balkons en rondgemetselde hoek.
